In [ ]:
# research_agent_gemini.py
import os
from dotenv import load_dotenv
from agno.agent import Agent
from agno.models.google import Gemini
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools.newspaper4k import Newspaper4kTools
from agno.tools.reasoning import ReasoningTools
from typing import List, Dict
from dotenv import load_dotenv
import requests
import json

load_dotenv()

True

In [18]:
class AgnoAgent:
    def __init__(self):
        self.model = Gemini(
            id="gemini-2.0-flash",
            api_key=os.getenv("GEMINI_API_KEY")
        )
        
        self.prompt = """
            You are a world-class chess news curator, renowned for your insightful and engaging reporting. Your task is to provide the latest and most relevant news from the world of chess.

            Your primary goal is to deliver news that meets the highest journalistic standards.  Adhere to the following principles:

            * **Relevancy:** Ensure the news is directly related to chess (e.g., tournaments, matches, player profiles, rule changes, major announcements). Avoid tangential or marginally related information.
            * **Accuracy:** Verify the facts presented in the news.  Cross-reference information from multiple reliable sources to ensure correctness.  Do not include rumors or unverified claims.
            * **Comprehensiveness:** Go beyond simple reporting.  Provide sufficient context and background information to help the reader understand the significance of the news.  Summarize complex events clearly and concisely.
            * **Objectivity:** Present the news in a neutral and unbiased manner.  Avoid expressing personal opinions or taking sides in controversies.  If controversies are reported, present all sides fairly.
            * **Clarity and Style:** Write in clear, concise, and grammatically correct English.  Use a professional and engaging tone.  Avoid jargon or overly technical language, unless it is clearly explained.  The phrasing should be similar to that used by major news outlets like the Associated Press, Reuters, or ESPN.
            * **Timeliness:** Prioritize recent events.  Only include news published within the last 10 days.  Give preference to very recent news.
            * **Source Reliability:** Only use information from reputable sources, such as official tournament websites, FIDE announcements, established chess news publications (e.g., ChessBase, Chess.com), and major news organizations.  Do not use information from personal blogs, forums, or social media unless it can be verified from a more reliable source.

            Specifically, for each news item, provide the following:

            * **Headline:** A concise and attention-grabbing headline that accurately reflects the content of the news.
            * **Summary:** A detailed summary of the news, including all the essential information.  The summary should be well-structured, easy to read, and approximately 150-200 words in length.
            * **Source Link**: Include the link to the original article.

            provide your response only as an object with the fields: title (representing the headline), description (representing the summary), source_links (array of links even if its a single link).

            Your output should be suitable for a discerning audience of chess enthusiasts, including players, coaches, and commentators.
            """

    def get_news(self):

        agent = Agent(
            model=self.model,
            tools=[
                DuckDuckGoTools(),
                Newspaper4kTools(),
                ReasoningTools(),
            ],
            instructions=[self.prompt],
            markdown=True
        )

        response = agent.run("Get the most recent and important chess news.")

        return json.loads(response.content[7:-3])


In [ ]:

def translate_text(source: Dict[str, str], target_languages: List[str]) -> List[Dict[str, any]]:

    url = "https://api.sarvam.ai/translate"
    target_language_codes = ["en-IN", "hi-IN", "bn-IN", "gu-IN", "kn-IN", "ml-IN", "mr-IN", "od-IN", "pa-IN", "ta-IN", "te-IN"]
    translations = []

    for target_lang in target_languages:
        if target_lang not in target_language_codes:
            print(f"Unsupported language code: {target_lang}")
            continue
        
        translation = {}
        for key in source.keys():
            if key == 'source_links':
                continue
            payload = {
                "input": source[key],
                "source_language_code": "auto",
                "target_language_code": target_lang,
                "speaker_gender": "Male",
                "mode": "formal",
                "model": "mayura:v1",
                "enable_preprocessing": False,
                "output_script": "roman",
                "numerals_format": "international",
                "output_script": "fully-native"
            }
            headers = {'api-subscription-key': os.getenv("SARVAM_API_KEY")}

            response = requests.request("POST", url, json=payload, headers=headers)
            if response.status_code == 200:
                translation[key] = (response.json())["translated_text"]
            else:   
                print(f"Error: {response.status_code}")
                print(response.text)
        
        translation['target_language_code'] = target_lang
        translations.append(translation)

    return translations


In [20]:
agno_agent = AgnoAgent()
response = agno_agent.get_news()

In [22]:
original_news = response
print(json.dumps(original_news, indent=4))

{
    "title": "Hans Niemann Sues Chess.com, Magnus Carlsen, and Others for Defamation",
    "description": "Hans Niemann has filed a lawsuit against Chess.com, Magnus Carlsen, Hikaru Nakamura, Danny Rensch, and others, alleging defamation and collusion. The lawsuit stems from accusations of cheating that arose after Niemann defeated Carlsen at the Sinquefield Cup in September 2022. Niemann is seeking at least $100 million in damages, claiming that the defendants conspired to damage his reputation and career. The suit alleges that Carlsen publicly accused him of cheating, while Chess.com banned him from their platform and removed his content. Nakamura is accused of amplifying the cheating accusations on his streaming platforms. The lawsuit brings further attention to the ongoing controversy surrounding Niemann and the issue of cheating in chess.",
    "source_links": [
        "https://www.chess.com/news/view/hans-niemann-sues-chess-com-magnus-carlsen"
    ]
}


In [35]:
translated_text = translate_text(original_news, ["hi-IN"])

In [41]:
news_with_translations = {
    "title": original_news["title"],
    "description": original_news["description"],
    "source_links": original_news["source_links"],
    "translations": translated_text
}

print("news: ", news_with_translations)

news:  {'title': 'Hans Niemann Sues Chess.com, Magnus Carlsen, and Others for Defamation', 'description': 'Hans Niemann has filed a lawsuit against Chess.com, Magnus Carlsen, Hikaru Nakamura, Danny Rensch, and others, alleging defamation and collusion. The lawsuit stems from accusations of cheating that arose after Niemann defeated Carlsen at the Sinquefield Cup in September 2022. Niemann is seeking at least $100 million in damages, claiming that the defendants conspired to damage his reputation and career. The suit alleges that Carlsen publicly accused him of cheating, while Chess.com banned him from their platform and removed his content. Nakamura is accused of amplifying the cheating accusations on his streaming platforms. The lawsuit brings further attention to the ongoing controversy surrounding Niemann and the issue of cheating in chess.', 'source_links': ['https://www.chess.com/news/view/hans-niemann-sues-chess-com-magnus-carlsen'], 'translations': [{'title': 'हंस नीमैन ने चेस ड